<a href="https://colab.research.google.com/github/anqi-guo/duplicated_complaints_identification/blob/main/3_process_labeled_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import os

path = '/content/drive/My Drive/work/重复线索识别/labeled_data_xlsx'

df_list = []
for file in os.listdir(path):
  dff = pd.read_excel(f'{path}/{file}')
  dff.columns = ['label', 'sentence1', 'sentence2']
  df_list.append(dff)

df = pd.concat(df_list)
df.head()

In [ ]:
df = df[~((df['sentence1'].str.contains('前期工单|原工单|已有工单|前单号'))|(df['sentence2'].str.contains('前期工单|原工单|已有工单|前单号')))]
df.shape

In [ ]:
# 删掉重复句子对
def sort_list(sent1, sent2):
  sents_list = list(sent1) + list(sent2)
  sents_list.sort()
  return ''.join(sents_list)

df['word_list'] = df.apply(lambda row: sort_list(row['sentence1'],row['sentence2']), axis=1)

df.drop_duplicates(subset=['word_list'], keep='first', inplace=True)

df.shape

In [ ]:
import regex as re
import jieba

def extract_problem(text):
  text = re.sub(r'\p{P}+', '', text) # 删掉标点符号
  text = re.sub(r'\s', '', text) # 删掉空格
  text = re.sub(r'\n', '', text)

  if '问题描述' in text and '诉求目的' in text:
    result = re.search(r'(?<=问题描述).*(?=诉求目的)', text).group()

  elif text.startswith('市民来电反映'):
    result = text.replace('市民来电反映', '')
    
  else:
    result = text
  
  result = re.sub('信息.{0,3}保密', '', result) # 删掉"信息保密"或"信息不保密"
  result = re.sub(r'\d+[米m]', '', result) # 删掉距离信息
  result = re.sub(r'\d+(年|月|月份|日)', '', result) # 删掉时间信息
  result = re.sub(r'(.{1}先生|.{1}女士)|市民)(来电)?反映', '', result) # 删掉投诉人信息

  # 删掉停用词
  with open('/content/drive/My Drive/work/重复线索识别/stopwords/cn_stopwords.txt') as f:
    stopwords = f.read().splitlines() 

  seg_list = jieba.cut(result)

  result = ''.join([s for s in seg_list if s not in stopwords])

  return result

df['sentence1'] = df['sentence1'].apply(extract_problem)
df['sentence2'] = df['sentence2'].apply(extract_problem)

In [ ]:
df.to_excel('/content/drive/My Drive/work/重复线索识别/data/processed_data/labeled_data/final_data.xlsx', index=False)